In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Flag to print the various intermediate data sets or skip them altogether
fPrint = True

df = pd.read_csv("/kaggle/input/d/anandhuh/latest-covid19-india-statewise-data/Latest Covid-19 India Status.csv")
df1 = pd.read_csv("/kaggle/input/indianstates-rulinggovernment-population-area/StateGovernmentsPopulationArea.csv")
# df[df['State/UTs'].str(0) == 'D'].head()
if fPrint:
    print(df.head())
    print(df1.head())

#### Adding Population, Ruling Government and Geographical Area data to the dataset to make it rich

In [ ]:
df.set_index('State/UTs', inplace=True)
df1.set_index('StatesUT', inplace=True)

df[['GovCat','Pop','Area']] = df1[['Gov Cat', 'Population2011', 'AreaSqKM']]
if fPrint:
    df.GovCat.value_counts()

##### Renaming Columns to ensure easy access to them in calculations

In [ ]:
df.rename(columns={'Death Ratio (%)': 'DeathRatio', 
                  'Active Ratio (%)': 'ActiveRatio',
                  'Total Cases': 'TotalCases',
                  'Discharge Ratio (%)': 'DischargeRatio'
#                   ,'State/UTs': 'StateUT'
                  }, inplace=True)
if fPrint:
    df.head()

In [ ]:
# Searching in the Dataset > Exploring the data. Finding the states starting with 'D'
if fPrint:
    df[df.index.str[0] == 'D'].head()

##### Highlighting the Top and Bottom 3 States/UTs in the terms of Death Ratio. A good representation of the management of the State

In [ ]:
df_sortedDR = df.sort_values('DeathRatio', ascending=False)
df_sortedDR_TopBottom = df_sortedDR.head(3).append(df_sortedDR.tail(3))
ls = df_sortedDR.head(3).append(df_sortedDR.tail(3))
ls

#### State wise Analysis of Geographical Area, Population and Covid Cases. Not giving very stark observations though

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
plt.rcParams['figure.figsize'] = (10,30)
plt.plot(df.Area, df.index, label="Geographical Area")
plt.plot(df.Pop/1000, df.index, label="Population")
plt.plot(df.TotalCases/10, df.index, label="Covid Cases")

plt.legend()
plt.show()

In [ ]:
group = df.groupby('GovCat').sum()
if fPrint:
    group

In [ ]:
group['CasesPercentOfPopulation'] = group['TotalCases']/group['Pop'] * 100
group['DeathRatio'] = group['Deaths'] / group['TotalCases'] * 100
govStat = group[['CasesPercentOfPopulation','DeathRatio']]
govStat

In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
plt.bar(govStat.index, govStat.CasesPercentOfPopulation, color='red')
plt.title('Percentage Cases of Total Population by Gov in States')
plt.show()
plt.title('Death Ratio (%) by Gov in States')
plt.bar(govStat.index, govStat.DeathRatio, color='maroon')